# CSE 6242 Spring 2021 - Team 4  
### Georgia Institute of Technology
### J. Carbonell, C. O’Brien, N. Ramallo, A. Sharma, I. Tumey, W. Wirono 

This notebook contains the data preparation and visualization our final project for Georgia Tech's CSE 6242 class in the Spring of 2021. To contextualize this notebook, we've included the introduction and problem statement from our report below.

> **Introduction:** By identifying where crimes are likely to occur and restructuring patrol routes to increase police presence in these areas, place-based predictive policing technologies offered the promise of deterring rather than responding to crimes. They also promised more “objective” direction of police activity – pushing police to where crime occurs rather than being guided racist notions of which neighborhoods harbor criminals. Proponents of these technologies often overlook the extent to which crime data, on which the models are trained, are themselves an artefact of racist historical patterns of policing.

> **Problem Statement:** We aim to develop granular maps of crime predictions and neighborhood demographics to evaluate the extent to which place-based crime prediction is racially biased. We approximate predictive policing techniques by using time-series forecasting and calls for service data from Baltimore, MD, to predict where crime is most likely to occur on an hourly basis – crime “hot spots”. To investigate the influence of data inputs on which neighborhoods (and racial groups, by extension) are identified as hot spots, we compare predictions based on calls for service and crime data.

> Understanding the difference between calls for service and crime is key to appreciating the value of the proposed approach. A call for service is generated when a Baltimore resident calls 911 or police internally request the assistance of additional officers for public safety issues, representing a broad range of potential police activity. Not every call for service results in a crime being reported or an arrest being, nor does every crime reported correspond to a call for service. The production processes for each type of data are therefore distinct. We investigate whether these distinct types of data generate different predictions of where and when crime will occur, as well as the implications for what neighborhoods are policed more heavily.

## Data Prep

We'll start by importing the packages neccesary for this notebook.

In [1]:
import branca.colormap as cm
import folium 
from geojson import dump
import json
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon, shape
from statsmodels.tsa.arima.model import ARIMA 

### Loading the Data

Next, we'll load three data files: crime data in Baltimore in 2018, ccalls for service (CFS) data in Baltimore in 2018, and demographic data for each [Census Tract](https://en.wikipedia.org/wiki/Census_tract#United_States) in [Baltimore](https://planning.baltimorecity.gov/sites/default/files/Neighborhood%20Statistical%20Areas%20with%20Census%20Tracts.pdf). The source of these files and the pre-processing that has been applied is specified in README.txt.

In [2]:
# Loading the crime data
crime_data = pd.read_csv('Data/crime_data_baltimore_2018.csv',)

# Prepping the date for prediction forecasting
crime_data.crimeDateTime = pd.to_datetime(crime_data.crimeDateTime).dt.date

In [3]:
# Loading the calls for service data
cfs_data = pd.read_csv('Data/calls_for_service_data_baltimore_2018.csv')

# Prepping the date for prediction forecasting
cfs_data.callDateTime = pd.to_datetime(cfs_data.callDateTime).dt.date

In [4]:
# Loading the Baltimore Census Tract demographic data
demog_data = pd.read_csv('Data/census_demographic_data.csv')

### Loading the GeoJSON data

We need to prep our GeoJSON data for the visualization. We'll start by importing a GeoJSON with the boundaries for ~450 identical geohashes within Baltimore. We'll also load a GeoJSON with the Baltimore census tract boundaries.

In [5]:
# Load the GeoJSON data
with open("Data/baltimore_geohash.json") as jsonfile: # GeoJSON for ~450 geohashes in Baltimore (for crime)
    geo_hash = json.load(jsonfile)
with open("Data/baltimore_geohash.json") as jsonfile: # GeoJSON for ~450 geohashes in Baltimore (for CFS)
    geo_hash2 = json.load(jsonfile)
with open("Data/census_tract_boundaries-demog.geojson") as jsonfile: # GeoJSON for Baltimore census tract boundaries
    census_geodata = json.load(jsonfile)

Next, we'll create lists with the name and location of each geohash.

In [6]:
# Looping through the geohash GeoJSON file and extracting the name and coordinates of each Geohash
geo_locs = []
geo_polys = []
for i in range(len(geo_hash['features'])):
    geo_polys.append(geo_hash['features'][i]['geometry']['coordinates'][0])
    geo_locs.append(geo_hash['features'][i]['properties']['geohash']) 

We'll use the lists above to find the census tract that has the most overlap with each geohash. We'll do this so we can connect each geohash to the demographic data of the census tract it resides in.  

In [7]:
# Find the census tract with the most overlap to each geohash
geohash_to_census = {}
for i in range(len(geo_locs)):
    lst = []
    p = Polygon(geo_polys[i])
    # Compute the overlapping area with each census tract
    for j in range(len(census_geodata['features'])):
        q = Polygon(census_geodata['features'][j]['geometry']['coordinates'][0])
        lst.append(p.intersection(q).area)        
    # Add max area census tract to JSON data  
    tract = census_geodata['features'][np.argmax(lst)]['properties']['NAME_y']
    geohash_to_census[geo_locs[i]] = tract

### Preparing the data for the visualization

We will forecast crime/CFS for each geohash using a time-series forecasting technique called [ARIMA](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average). A series of experiments generated
ARIMA(3,0,1) as the best fit model for both the crime and calls for service data. To validate the model, an ARIMA library titled [pmdarima](https://pypi.org/project/pmdarima/) was used. We took advantage of the stepwise grid-search that minimizes the AIC. For a 30-day prediction timeframe, the crime model has an MSE of 562.99 and the calls for service model generates an MSE of 19,041.24. 

First, we'll fit an ARIMA model to both datasets.

In [8]:
# Fitting ARIMA model to crime data
crime_freq = crime_data.groupby("crimeDateTime").count().reset_index(False)
crime_freq['Count'] = crime_freq['OBJECTID']
crime_freq = crime_freq.set_index('crimeDateTime').asfreq('d')
model_crime = ARIMA(crime_freq['Count'], order=(3,0,1),freq='D')
model_crime_fit = model_crime.fit()

In [9]:
# Fitting ARIMA model to CFS data
cfs_freq = cfs_data.groupby("callDateTime").count().reset_index(False)
cfs_freq['Count'] = cfs_freq['OBJECTID']
cfs_freq = cfs_freq.set_index('callDateTime').asfreq('d')
model_cfs = ARIMA(cfs_freq['Count'], order=(3,0,1),freq='D')
model_cfs_fit = model_cfs.fit()

#### Function to forecast crimes/CFS
Next, we'll define a function that takes in a dataset and a geohash and returns a crime/CFS prediction for that geohash on January 1st, 2019 (i.e. one day after the dataset ends).

In [10]:
import warnings
warnings.filterwarnings('ignore')

# ARIMA prediction function
def get_prediction(data,geohash):
    d = data[data.geohash == geohash] # get data for only that geohash
    if len(d)==0:
        return 0
    elif len(d)<100: # if sample size is too small, return the mean
        return len(d)/365
    if 'crimeDateTime' in data.columns: # If crimedata
        crime_freq = d.groupby("crimeDateTime").count().reset_index(False) # Get count data
        crime_freq = crime_freq.set_index('crimeDateTime').asfreq('d') # set index to date
        pred = model_cfs_fit.apply(crime_freq['OBJECTID']).forecast(1) # forecast
    else: # if cfs data
        crime_freq = d.groupby("callDateTime").count().reset_index(False)  # Get count data
        crime_freq = crime_freq.set_index('callDateTime').asfreq('d') # set index to date
        pred = model_crime_fit.apply(crime_freq['OBJECTID']).forecast(1) # forecast
    return float(pred)

#### Function to add demographic data to a GeoJSON

To display the demographic data for a geohash in the tooltip, we'll define a function that takes in a GeoJSON and a census tract, then adds the demographic data for that census tract to the GeoJSON.

In [11]:
def add_demographic_data(ghash,n):
    ghash1 = ghash.copy()
    df = demog_data[demog_data.NAME==n] # Get demographic data for that Census Tract
    df = df.replace('-',0)
    ghash1['Total Pop'] = int(df.Total_pop)
    ghash1['White Pop'] = int(df.White_pop)
    ghash1['White Pct'] = str(float(df.White_pct))+'%'
    ghash1['Black Pop'] = int(df.Black_pop)
    ghash1['Black Pct'] = str(float(df.Black_pct))+'%'
    ghash1['Latino Pop'] = int(df.Latino_pop)
    ghash1['Latino Pct'] = str(float(df.Latino_pct))+'%'
    ghash1['Asian Pop'] = int(df.Asian_pop)
    ghash1['Asian Pct'] = str(float(df.Asian_pct))+'%'
    ghash1['Pct Below Pov Level'] = str(float(df.Pct_below_pov_level))+'%'
    return ghash1    

#### Function to get crime and CFS counts by census tract and connect it to a GeoJSON

Let's define a function that connects what we've done above. It'll take in a dataset and a GeoJSON, then:
- Create a dictionary with the model prediction for each geohash.
- Update the GeoJSON with the census tract for each geohash.
- Update the GeoJSON with the demographic data for each geohash (based off it's census tract).

In [12]:
def get_counts(data,ghash,ret_geodata=True):
    ghash = ghash.copy()
    dct = {}
    # Get prediction for each geohash
    for i in geo_locs:
        dct[i] = get_prediction(data,i)
    # Set prediction for each geohash equal to the proportion of all values
    sum_pred = sum(dct.values())
    dct = {k: v / sum_pred*100 for k, v in dct.items()}
    # Option to return just the prediction dictionary
    if not ret_geodata:
        return dct
    # Looping through each geohash and adding the crime/CFS prediction, census tract, and demographic data
    else:
        for i in range(len(ghash['features'])):
            # Get that polygon's geohash
            n = ghash['features'][i]['properties']['geohash']
            # Assign the census tract for that geohash
            ct = geohash_to_census[n]
            ghash['features'][i]['properties']['Census Tract'] = ct
            ghash['features'][i]['properties']['Census Tract Short'] = ct[13:]
            # Add predicted proportion to geojson
            if n in dct.keys():
                ghash['features'][i]['properties']['pred_prop'] = dct[n]
                ghash['features'][i]['properties']['pred_prop_perc'] = str(round(dct[n],4)) + '%'
            else: # If no data exist for that geohash, set count to 0.
                ghash['features'][i]['properties']['pred_prop'] = 0
                ghash['features'][i]['properties']['pred_prop_perc'] = 0
            # Add demographic data to json
            ghash['features'][i]['properties'] = add_demographic_data(ghash['features'][i]['properties'],ct)
        return ghash

#### Final data prep

We now are ready to call the 'get_counts' function on both the crime and CFS datasets.

In [13]:
data_crime = get_counts(crime_data,geo_hash)
data_cfs = get_counts(cfs_data,geo_hash2)

### Visualization Prep

#### Defining and applying a color scale

Next, we'll define a color map to visualize the differences in the predicted proportion of crime/CFS by geoash. We'll also define a style function for Folium that colors a geohash (using the color map) according to its forecasted crime/CFS amount.

In [14]:
# Defining a colormap
colors = ['#fff5f0','#fee0d2','#fcbba1','#fc9272','#fb6a4a','#ef3b2c','#cb181d','#a50f15','#67000d']
scale_index = [0.0,.2,.3,.4,.5,0.6,.7,.8,1.0]
colormap = cm.StepColormap(colors, index = scale_index)
# Defining a style function to color a geohash based of it's forecasted crime/CFS proportion
def style_function(feature): 
    co = colormap(feature['properties']['pred_prop'])
    return {'fillColor': co,"fillOpacity": 0.5, 'color':'white'}

#### Defining tooltips

We can define a custom tooltip that leverages the features for each geohash in the GeoJSON files. Folium prefers that layers don't share tooltips, so we'll define an identical tooltip for each layer.

In [15]:
# Defining a tooltip (crime)
tooltip1 = folium.features.GeoJsonTooltip(
    fields=['Census Tract Short','pred_prop_perc','White Pct','Black Pct','Latino Pct','Asian Pct'],
    aliases=['Census Tract','Pred Crime Proportion','White Pct','Black Pct','Latino Pct','Asian Pct'],
    localize=True,sticky=False,labels=True,max_width=800,direction = 'left',
    style=""" padding: 8px; background-color: white; border-radius: 2px; font-size: 12px;
              font-family: Roboto,sans-serif; font-weight: 400;""")
# Defining a tooltip (cfs)
tooltip2 = folium.features.GeoJsonTooltip(
    fields=['Census Tract Short','pred_prop_perc','White Pct','Black Pct','Latino Pct','Asian Pct'],
    aliases=['Census Tract','Pred CFS Proportion','White Pct','Black Pct','Latino Pct','Asian Pct'],
    localize=True,sticky=False,labels=True,max_width=800,direction = 'left',
    style=""" padding: 8px; background-color: white; border-radius: 2px; font-size: 12px;
              font-family: Roboto,sans-serif; font-weight: 400;""")

## Creating the Visualization

With our data prepped, we can now use Folium to produce our map. First, we'll create a map object that's centered around Baltimore.

In [16]:
m = folium.Map(location=[39.2904, -76.6122],zoom_start=11.4,tiles=None)

We'll create two map layers, one for the crime data and another for the calls for service data.

In [17]:
# Feature layer1 - crime
feature_group1 = folium.FeatureGroup(name="Crime Prediction",overlay=False,show=True).add_to(m) # Creating feature group
feature_group1.add_child(folium.raster_layers.TileLayer()) # Adding map background
feature_group1.add_child(folium.GeoJson(data_crime, name="geojson_crime",style_function = style_function, tooltip = tooltip1)); # Adding geojson data

#Feature layer2 - CFS
feature_group2 = folium.FeatureGroup(name="Calls for Service Prediction",overlay=False,show=False).add_to(m) # Creating feature group
feature_group2.add_child(folium.raster_layers.TileLayer()) # Adding map background
feature_group2.add_child(folium.GeoJson(data_cfs, name="geojson2_cfs",style_function = style_function, tooltip = tooltip2)); # Adding geojson data

Next, we'll add the color scale to the map, as well as a layer control that allows the user to switch between the crime and CFS maps.

In [18]:
colormap.add_to(m) # Adding color map to map
folium.LayerControl(collapsed=False).add_to(m); # Adding layer control to map

Lastly, we can call the map to view the visualization.

In [19]:
m